# Markov Chain Solver Instructions 

This tool takes a probability transition matrix and an initial state vector and outputs a diagram, the steady state vector, and a plot showing all iterations through the system before reaching the steady state. 

The following parameters can be toggled individually:
* n_random      (number of states)
* prob_random   (probability transition matrix)
* state_random  (initial state vector)

Toggle a parameter to **"1"** if you want to **randomize** it. Toggle a parameter to **"0"** if you want to **customize** it. 

Customization Constraints:
 
* **prob_custom** must be an **(n x n) np.array**. All rows must sum to **1**.
* **state_custom** must be a **(1 x n) np.array**. Elements must sum to **1**.
* **n_random** must be set to **"0"** if you are customizing **state_specific** or **prob_specific**. **n_specific** must agree with their dimensions.
* **sparse** must be between **0** and **0.7**. Otherwise it will take too many tries to make a well-connected chain.

After choosing your inputs in the cell labeled **"User Inputs"**, press **"Run"** to start. The completed figure will be generated at the bottom of the notebook.

*Notebook written by Jared Rogers on December 15, 2020.*

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym
import string

In [2]:
## User Inputs

# Toggle automatic nodes, probabilities, initial state (1 = randomized, 0 = customized by user in the lines below)
n_random = 0
prob_random = 1
state_random = 1

# Adjust Sparseness (reduces the number of overall connections in the chain) (0 = no reduction, don't exceed 0.7)
sparse = 0.3 

# Input specific nodes, probabilities, initial state
n_specific = 5                                              #must be between 2 and 8 and agree with prob_specific and state_specific
prob_specific = np.array([[2/3, 1/3, 0. , 0. , 0. ],             #must be (n x n) np.array with each row summing to 1
                          [1/5, 1/5, 1/5, 1/5, 1/5],
                          [1/3, 1/3, 0. , 1/3, 0. ],
                          [0. , 4/6, 0. , 1/6, 1/6],
                          [0. , 0. , 1/2, 0. , 1/2]])
state_specific = np.array([1, 0, 0, 0, 0])                 #must be (1 x n) np.array summing to 1

if ((n_random == 0) & ((n_specific < 2) | (n_specific > 8))):
    raise SystemExit("The value of n_specific must be an integer between 2 and 8! Alternatively, just set n_random to '1'.")

In [3]:
## Apply Inputs

# Assign either random n or specific n
if n_random == 1:
    n = np.random.randint(2,9);
else:
    n = n_specific;

# Attempt to assign probability matrix and retry up to 10 times if division by 0 occurs 
p = np.ones([9,1])*(1-sparse)/9
if prob_random == 1:
    for x in range(0, 10): 
        prob = np.zeros([n,n]);
        for i in np.arange(0,n):
            for j in np.arange(0,n):
                prob[i][j] = np.random.choice(np.arange(10), p = np.append(sparse,p)) ;
            prob_sum = np.sum(prob[i]);
            prob[i] = prob[i]/prob_sum;
            if prob_sum == 0:
                break
        if prob_sum == 0:
            pass
        else:
            break
else:
    prob = prob_specific;

# Attempt to assign initial state vector and retry up to 10 times if division by 0 occurs 
if state_random == 1:
    for x in range(0, 10): 
        state = np.zeros([n,1]);
        for i in np.arange(0,n):
            state[i] = np.random.randint(5);
        state_sum = np.sum(state);
        state = (state/state_sum).T;
        state = state[0];
        if state_sum == 0:
            pass
        else:
            break
else:
    state = np.array(state_specific);

In [4]:
## Create Figure and All Axes

# Initialize figure
fig = plt.figure(figsize = [18,18], frameon = True, edgecolor = "k");
gs = fig.add_gridspec(6, 6);
ax0 = fig.add_subplot(gs[:4,:4]);
ax1 = fig.add_subplot(gs[:2,-2:]);
ax2 = fig.add_subplot(gs[-4,-2:]);
ax3 = fig.add_subplot(gs[-3,-2:]);
ax4 = fig.add_subplot(gs[-2:,:]);
winsize = 7;

# Set up Transition Diagram subplot
ax0.set_title("Transition Diagram", fontsize = 24, pad = -25, fontweight = "bold")
ax0.set_xlim([-winsize/2,winsize/2]);
ax0.set_ylim([-winsize/2,winsize/2]);
ax0.xaxis.set_visible(False);
ax0.yaxis.set_visible(False);

# Set up Probability Transition Matrix subplot
ax1.set_title("Probability Transition \n Matrix", fontsize = 24, pad = -54, fontweight = "bold")
ax1.set_xlim([-winsize/4,winsize/4]);
ax1.set_ylim([-winsize/4,winsize/4]);
ax1.xaxis.set_visible(False);
ax1.yaxis.set_visible(False);

# Set up Initial State Vector subplot
ax2.set_title("Initial State Vector", fontsize = 24, pad = -25, fontweight = "bold")
ax2.set_xlim([-winsize/4,winsize/4]);
ax2.set_ylim([-winsize/8,winsize/8]);
ax2.xaxis.set_visible(False);
ax2.yaxis.set_visible(False);

# Set up Steady State Vector subplot
ax3.set_title("Steady State Vector", fontsize = 24, pad = -25, fontweight = "bold")
ax3.set_xlim([-winsize/4,winsize/4]);
ax3.set_ylim([-winsize/8,winsize/8]);
ax3.xaxis.set_visible(False);
ax3.yaxis.set_visible(False);

# Set up Convergence of Steady State Subplot
ax4.set_title("Convergence of Steady State", fontsize = 24, pad = -25, fontweight = "bold", backgroundcolor = "w")
ax4.set_xlabel("Iteration", fontsize = 20, labelpad = 18)
ax4.set_ylabel("Probability", fontsize = 20, labelpad = 18)
ax4.set_xlim([0,100]);
ax4.set_ylim([0,1]);
ax4.tick_params(axis = "x", labelsize = 15, size = 5)
ax4.tick_params(axis = "y", labelsize = 15, size = 5)

plt.close()

In [5]:
## Solve Transition Matrix Steady State Analytically

# Define list of variables (unknown steady state elements)
steady_var = sym.symbols("a0:" + str(n))

# Define normalization condition (sum of steady state elements must equal 1)
norm_cond = np.sum(np.asarray(steady_var)) - 1

# Define system of linear equations including all relevant conditions
system_part = np.dot(steady_var,(prob - np.identity(n)))
system_full = np.append(system_part,norm_cond)

# Solve system of linear equations and convert output to float array
for i in np.arange(0,n):
    steady_solve = sym.solve(system_full,steady_var)
steady_solve = list(steady_solve.values())
steady_solve = np.asarray(steady_solve, dtype = "float")
steady_solve = np.around(steady_solve,3)
steady_solve = steady_solve[::-1]

In [6]:
## Plot Nodes and Vertices of Markov Chain Graph

# Set up list of node labels
letter = list(string.ascii_uppercase);

# Define colors for nodes
cmap = plt.cm.gist_rainbow(np.linspace(0,1,n+1));

# Plot evenly spaced nodes and connect with edges
r = 0.6 * winsize/2;
angle = 2 * np.pi/n;
xy = np.zeros([n,2]);

for i in np.arange(0,n):
    xy[i][0] = np.around(r * np.cos(np.pi/2 - angle * i),3);
    xy[i][1] = np.around(r * np.sin(np.pi/2 - angle * i),3) - 0.25;
    ax0.add_artist(matplotlib.patches.Circle(xy[i], radius = winsize*0.3/n, linewidth = 10*prob[i][i]/np.max(prob),
                                             edgecolor = 'k', facecolor = cmap[i], alpha = 1));
    ax0.scatter(xy[i][0]-0.075/n, xy[i][1], marker = "$\mathrm{" + letter[i] + "}$", 
                s = 25000 / np.square(n), c = 'k', zorder = n);
    for j in np.arange(0,n):
        if prob[i][j] != 0:
            ax0.add_artist(matplotlib.patches.FancyArrowPatch(xy[i],xy[j], connectionstyle = "Arc3, rad =" + str(1/n),
                                                              zorder = 0, arrowstyle = "-", color = cmap[i], 
                                                              linewidth = 15 * prob[i][j] / np.max(prob)));

In [7]:
## Iterate Initial State Vector through Markov Chain "k" Times and Plot Evolution

# Set number of iterations to attempt
k = 50

# Repeatedly apply transition matrix to state vector until it stabilizes
state_iter = np.zeros([k,n])
state_iter[0][:] = state
for i in np.arange(1,k):
    state_iter[i] = np.matmul(state_iter[i-1].T, prob).T
    if all(np.around(state_iter[i],5) == np.around(state_iter[i-1],5)):
        k_stop = i
        state_iter = state_iter[:k_stop,:]
        break

# Plot each iteration of state vector elements
for i in np.arange(0,n):
    ax4.plot(np.arange(0,k_stop),state_iter[0:k_stop,i], lw = 4, c = cmap[i])

ax4.legend(list(letter[0:n]), loc = "upper right", fontsize = 15, ncol = 2, framealpha = 1)
ax4.set_xlim([0,k_stop - 1])
ax4.set_ylim([0,np.max(state_iter) + 0.1])
ax4.tick_params(axis = "both", labelsize = 15, size = 5)

# Plot vertical lines at each iteration
for i in np.arange(1,k_stop-1):
    ax4.axvline(x = i, color = [0.9,0.9,0.9], zorder = 0)

plt.close()

In [8]:
## Generate LaTeX Transition Matrix

# Adjust rcParams to allow for LaTeX integration
matplotlib.rcParams['font.weight'] = "bold"
matplotlib.rcParams['font.size'] = 90/n
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{{amsmath}}'

# Create LaTeX code for transition matrix 
trans_mat_pretty = r'$\begin{bmatrix}'
for i in np.arange(0,n):
    for j in np.arange(0,n):
        trans_mat_pretty = trans_mat_pretty + str(np.around(prob[i][j],3))
        if (j != n-1): 
            trans_mat_pretty = trans_mat_pretty + r'\ &'
        elif ((j == n-1) & (i != n-1)):
            trans_mat_pretty = trans_mat_pretty + r'\\'            
trans_mat_pretty = trans_mat_pretty + r'\end{bmatrix}$'

# Create LaTeX code for initial state vector
state_pretty = r'$\begin{bmatrix}'
for i in np.arange(0,n):
    state_pretty = state_pretty + str(np.around(state[i],3))
    if (i != n-1):
        state_pretty = state_pretty + r'\ &'
state_pretty = state_pretty + r'\end{bmatrix}$'

# Create LaTeX code for steady state vector
steady_pretty = r'$\begin{bmatrix}'
for i in np.arange(0,n):
    steady_pretty = steady_pretty + str(np.around(steady_solve[i],3))
    if (i != n-1):
        steady_pretty = steady_pretty + r'\ &'
steady_pretty = steady_pretty + r'\end{bmatrix}$'

# Populate axes with LaTeX transition probability matrix, initial state vector, and steady state vector
ax1.text(0.05,-0.4, trans_mat_pretty, horizontalalignment = 'center', verticalalignment = 'center')
ax2.text(0.05,-0.2, state_pretty,     horizontalalignment = 'center', verticalalignment = 'center')
ax3.text(0.05,-0.2, steady_pretty,    horizontalalignment = 'center', verticalalignment = 'center')

fig.suptitle("--------- Markov Chain Solver ---------", fontsize = 60, y = 0.95, fontfamily = "DejaVu Sans");

fig

FileNotFoundError: [WinError 2] The system cannot find the file specified

<Figure size 1296x1296 with 5 Axes>

# Interpreting the Transition Diagram

Each **node** represents a **state** in the system. 

The **thickness** of the **black ring** around a node corresponds to its **probability of transitioning to itself**. 

Connections matching the **color** of a node represent a **transition *from* that node**. 

The **thicker** the connection, the **greater** its associated **transition probability**.

In [ ]:
# Save Figure (optional)

# Toggle "save" to "1" if you want to save the above figure.
save = 0

if save == 1:
    fig.savefig("Markov Chain Solver.png")